In [37]:
import pandas as pd
import numpy as np
import xlwt
import xlrd
from tkinter import filedialog
import tkinter as tk
import customtkinter as ctk
from customtkinter import filedialog
import funciones
import openpyxl
import dbfread
import glob
from pandastable import Table, TableModel # Importar clases de pandastable

### Subir reporte danilo

In [38]:
archivo = None
while True:
    archivo=funciones.seleccionar_archivo()
    print(archivo)
    if archivo:
        ruta_archivo = archivo.name
        try:
            
            #dbf = dbfread.DBF(ruta_archivo)
            monotributo = pd.read_excel(ruta_archivo, sheet_name=0)
            print(monotributo)
            break
        except UnicodeDecodeError:
            print("Error de decodificación de caracteres. Intenta con otra codificación o archivo.")
            continue
    else:
        print("Archivo incorrecto. Por favor, vuelva a cargarlo.")

<_io.TextIOWrapper name='G:/Mi unidad/osoetsylra/liquidaciones/cruce_contribuciones_para_distribuir/transferncias AFIP reporte de errores en importacion Gecros octubre 2023 completo.xlsx' mode='r' encoding='cp1252'>
             cuit  cuil                                    descripcion  \
0     20401847341   NaN  401: Contrib. Patr. sin Empl. para distribuir   
1     20401847341   NaN  401: Contrib. Patr. sin Empl. para distribuir   
2     20401847341   NaN  401: Contrib. Patr. sin Empl. para distribuir   
3     20148029165   NaN  401: Contrib. Patr. sin Empl. para distribuir   
4     20241060366   NaN  401: Contrib. Patr. sin Empl. para distribuir   
...           ...   ...                                            ...   
8666  27959478606   NaN  401: Contrib. Patr. sin Empl. para distribuir   
8667  27959819152   NaN  401: Contrib. Patr. sin Empl. para distribuir   
8668  27960117943   NaN  401: Contrib. Patr. sin Empl. para distribuir   
8669  27961039008   NaN  401: Contrib. Patr.

In [39]:
monotributo.dtypes

cuit                    int64
cuil                  float64
descripcion            object
obrasocial            float64
periodo                 int64
sueldo                  int64
aporte                  int64
contri                float64
otrosconc               int64
fecha          datetime64[ns]
dtype: object

###  Importar padron total gecros monotrbutistas

In [40]:
#cargar primero el padron de sidefi
archivo = None
while True:
    archivo = funciones.seleccionar_archivo()
    print(archivo)
    if archivo:
        ruta_archivo = archivo.name
        try:
            padron = pd.read_excel(ruta_archivo)
            #padron = pd.read_csv(archivo, sep=";", encoding="ISO-8859-1")
            print(padron)
            break
        except UnicodeDecodeError:
            print("Error de decodificación de caracteres. Intenta con otra codificación o archivo.")
            continue
    else:
        print("Archivo incorrecto. Por favor, vuelva a cargarlo.")

<_io.TextIOWrapper name='C:/Users/zickd/OneDrive/Escritorio/LIQUIDACION23.xlsx' mode='r' encoding='cp1252'>
                OS_NOMBRE          CUIL                    NOMPLAN PARENTESCO  \
0              OSOETSYLRA  2.092946e+10                  Afiliados    TITULAR   
1              OSOETSYLRA  2.436609e+10                   Plan 100    TITULAR   
2                 MEPLIFE  2.740914e+10  monotributista social mep    TITULAR   
3              OSOETSYLRA  2.727502e+10                Desempleado    TITULAR   
4              OSOETSYLRA  2.038062e+10                   Plan 100    TITULAR   
...                   ...           ...                        ...        ...   
66363  OTRAS AFILIACIONES  2.734093e+10                  Afiliados    TITULAR   
66364  OTRAS AFILIACIONES  2.736203e+10                  Afiliados    TITULAR   
66365             MEPLIFE  2.337400e+10  monotributista social mep    TITULAR   
66366  OTRAS AFILIACIONES  2.720696e+10                  Afiliados    TITULAR   
6

In [41]:
padron['CUIL'] = pd.to_numeric(padron['CUIL'], errors='coerce').fillna(0).astype('Int64')

In [42]:
padron['CUIL'].head()

0    20929460368
1    24366090777
2    27409138468
3    27275022255
4    20380615240
Name: CUIL, dtype: Int64

#### Cruce de archivos! 

In [43]:
liquidaciones_monot = pd.merge(monotributo, padron, left_on= 'cuit', right_on='CUIL', how='left', indicator=True)

In [47]:
path = funciones.seleccionar_directorio()
liquidaciones_monot.to_excel(path + '\cruce_errores.xlsx', index=False)

In [48]:
liquidaciones_monot.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8672 entries, 0 to 8671
Data columns (total 21 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   cuit         8672 non-null   int64         
 1   cuil         0 non-null      float64       
 2   descripcion  8672 non-null   object        
 3   obrasocial   0 non-null      float64       
 4   periodo      8672 non-null   int64         
 5   sueldo       8672 non-null   int64         
 6   aporte       8672 non-null   int64         
 7   contri       8672 non-null   float64       
 8   otrosconc    8672 non-null   int64         
 9   fecha        8672 non-null   datetime64[ns]
 10  OS_NOMBRE    6400 non-null   object        
 11  CUIL         6400 non-null   Int64         
 12  NOMPLAN      6400 non-null   object        
 13  PARENTESCO   6400 non-null   object        
 14  FECHA_BAJA   1982 non-null   object        
 15  EMP_NOMBRE   6394 non-null   object        
 16  APELLI

: 